In [7]:
# pip install requests pandas

import requests
import pandas as pd
import time

BASE_URL = "https://query2.finance.yahoo.com/v1/finance/screener/predefined/saved"

HDRS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/124.0.0.0 Safari/537.36",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://finance.yahoo.com/",
    "Connection": "keep-alive",
}

def fetch_page(session, start, count=100, timeout=15):
    params = {
        "count": count,
        "formatted": "false",
        "lang": "en-US",
        "region": "US",
        "scrIds": "day_gainers",
        "start": start,
    }
    # Simple retry loop
    for attempt in range(4):
        resp = session.get(BASE_URL, headers=HDRS, params=params, timeout=timeout)
        if resp.status_code == 200:
            return resp.json()
        time.sleep(1.2 * (attempt + 1))
    resp.raise_for_status()

def fetch_all_gainers(count_per_page=100, pause=0.8, max_pages=200):
    all_rows = []
    start = 0
    with requests.Session() as sess:
        # warm-up visit to set cookies
        sess.get("https://finance.yahoo.com/markets/stocks/gainers/", headers=HDRS, timeout=15)
        for _ in range(max_pages):
            data = fetch_page(sess, start, count_per_page)
            try:
                quotes = data["finance"]["result"][0]["quotes"]
            except (KeyError, IndexError, TypeError):
                break
            if not quotes:
                break
            all_rows.extend(quotes)
            start += count_per_page
            time.sleep(pause)
    return pd.DataFrame(all_rows)

if __name__ == "__main__":
    df = fetch_all_gainers()
    print(f"Retrieved {len(df)} rows")
    df.to_csv("yahoo_top_gainers.csv", index=False)
    print("Saved to yahoo_top_gainers.csv")

Retrieved 139 rows
Saved to yahoo_top_gainers.csv
